### Preprocessing data

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install pyvi

In [ ]:
import pandas as pd
import sklearn
import pyvi
import re
from pyvi import ViTokenizer

In [ ]:
df_train = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/Sentiment_BML/train.csv")
df_test = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/Sentiment_BML/test.csv")
df_train.head()

,comment,label
0,Mua có mỗi Bingsu thập cẩm 45k mà mình f đợi h...,0
1,Thứ 6 nào ta cùng quẩy 💣 💣 💣\r\n Vuvuzela beer...,0
2,"Mình đi với nhóm , tổng cộng 4 người ăn chỉ có...",0
3,"nhân viên phục vụ không mấy tận tình , đồ ăn r...",0
4,"Vào đây thì hết bàn , nhưng mình vẫn ngồi đợi ...",0


In [ ]:

def preprocessing(doc):
    replace_list = {
        'òa': 'oà', 'óa': 'oá', 'ỏa': 'oả', 'õa': 'oã', 'ọa': 'oạ', 'òe': 'oè', 'óe': 'oé','ỏe': 'oẻ',
        'õe': 'oẽ', 'ọe': 'oẹ', 'ùy': 'uỳ', 'úy': 'uý', 'ủy': 'uỷ', 'ũy': 'uỹ','ụy': 'uỵ', 'uả': 'ủa',
        'ả': 'ả', 'ố': 'ố', 'u´': 'ố','ỗ': 'ỗ', 'ồ': 'ồ', 'ổ': 'ổ', 'ấ': 'ấ', 'ẫ': 'ẫ', 'ẩ': 'ẩ',
        'ầ': 'ầ', 'ỏ': 'ỏ', 'ề': 'ề','ễ': 'ễ', 'ắ': 'ắ', 'ủ': 'ủ', 'ế': 'ế', 'ở': 'ở', 'ỉ': 'ỉ',
        'ẻ': 'ẻ', 'àk': u' à ','aˋ': 'à', 'iˋ': 'ì', 'ă´': 'ắ','ử': 'ử', 'e˜': 'ẽ', 'y˜': 'ỹ', 'a´': 'á',
        #Quy các icon về 2 loại emoj: Tích cực hoặc tiêu cực
        "👹": "negative", "👻": "positive", "💃": "positive",'🤙': ' positive ', '👍': ' positive ',
        "💄": "positive", "💎": "positive", "💩": "positive","😕": "negative", "😱": "negative", "😸": "positive",
        "😾": "negative", "🚫": "negative",  "🤬": "negative","🧚": "positive", "🧡": "positive",'🐶':' positive ',
        '👎': ' negative ', '😣': ' negative ','✨': ' positive ', '❣': ' positive ','☀': ' positive ',
        '♥': ' positive ', '🤩': ' positive ', 'like': ' positive ', '💌': ' positive ',
        '🤣': ' positive ', '🖤': ' positive ', '🤤': ' positive ', ':(': ' negative ', '😢': ' negative ',
        '❤': ' positive ', '😍': ' positive ', '😘': ' positive ', '😪': ' negative ', '😊': ' positive ',
        '?': ' ? ', '😁': ' positive ', '💖': ' positive ', '😟': ' negative ', '😭': ' negative ',
        '💯': ' positive ', '💗': ' positive ', '♡': ' positive ', '💜': ' positive ', '🤗': ' positive ',
        '^^': ' positive ', '😨': ' negative ', '☺': ' positive ', '💋': ' positive ', '👌': ' positive ',
        '😖': ' negative ', '😀': ' positive ', ':((': ' negative ', '😡': ' negative ', '😠': ' negative ',
        '😒': ' negative ', '🙂': ' positive ', '😏': ' negative ', '😝': ' positive ', '😄': ' positive ',
        '😙': ' positive ', '😤': ' negative ', '😎': ' positive ', '😆': ' positive ', '💚': ' positive ',
        '✌': ' positive ', '💕': ' positive ', '😞': ' negative ', '😓': ' negative ', '️🆗️': ' positive ',
        '😉': ' positive ', '😂': ' positive ', ':v': '  positive ', '=))': '  positive ', '😋': ' positive ',
        '💓': ' positive ', '😐': ' negative ', ':3': ' positive ', '😫': ' negative ', '😥': ' negative ',
        '😃': ' positive ', '😬': ' 😬 ', '😌': ' 😌 ', '💛': ' positive ', '🤝': ' positive ', '🎈': ' positive ',
        '😗': ' positive ', '🤔': ' negative ', '😑': ' negative ', '🔥': ' negative ', '🙏': ' negative ',
        '🆗': ' positive ', '😻': ' positive ', '💙': ' positive ', '💟': ' positive ',
        '😚': ' positive ', '❌': ' negative ', '👏': ' positive ', ';)': ' positive ', '<3': ' positive ',
        '🌝': ' positive ',  '🌷': ' positive ', '🌸': ' positive ', '🌺': ' positive ',
        '🌼': ' positive ', '🍓': ' positive ', '🐅': ' positive ', '🐾': ' positive ', '👉': ' positive ',
        '💐': ' positive ', '💞': ' positive ', '💥': ' positive ', '💪': ' positive ',
        '💰': ' positive ',  '😇': ' positive ', '😛': ' positive ', '😜': ' positive ',
        '🙃': ' positive ', '🤑': ' positive ', '🤪': ' positive ','☹': ' negative ',  '💀': ' negative ',
        '😔': ' negative ', '😧': ' negative ', '😩': ' negative ', '😰': ' negative ', '😳': ' negative ',
        '😵': ' negative ', '😶': ' negative ', '🙁': ' negative ',
        #Chuẩn hóa 1 số sentiment words/English words
        ':))': '  positive ', ':)': ' positive ', 'ô kêi': ' ok ', 'okie': ' ok ', ' o kê ': ' ok ',
        'okey': ' ok ', 'ôkê': ' ok ', 'oki': ' ok ', ' oke ':  ' ok ',' okay':' ok ','okê':' ok ',
        ' tks ': u' cám ơn ', 'thks': u' cám ơn ', 'thanks': u' cám ơn ', 'ths': u' cám ơn ', 'thank': u' cám ơn ',
        '⭐': 'star ', '*': 'star ', '🌟': 'star ', '🎉': u' positive ',
        'kg ': u' không ','not': u' không ', u' kg ': u' không ', '"k ': u' không ',' kh ':u' không ','kô':u' không ','hok':u' không ',' kp ': u' không phải ',u' kô ': u' không ', '"ko ': u' không ', u' ko ': u' không ', u' k ': u' không ', 'khong': u' không ', u' hok ': u' không ',
        'he he': ' positive ','hehe': ' positive ','hihi': ' positive ', 'haha': ' positive ', 'hjhj': ' positive ',
        ' lol ': ' negative ',' cc ': ' negative ','cute': u' dễ thương ','huhu': ' negative ', ' vs ': u' với ', 'wa': ' qua ', 'wá': u' quá', 'j': u' gì ', '“': ' ',
        ' sz ': u' cỡ ', 'size': u' cỡ ', u' đx ': u' được ', 'dk': u' được ', ' dc ': u' được ', 'đk': u' được ',
        'đc': u' được ','authentic': u' chuẩn chính hãng ',u' aut ': u' chuẩn chính hãng ', u' auth ': u' chuẩn chính hãng ', 'thick': u' positive ', 'store': u' cửa hàng ',
        'shop': u' cửa hàng ', 'sp': u' sản phẩm ', 'gud': u' tốt ','god': u' tốt ','wel done':' tốt ', 'good': u' tốt ', 'gút': u' tốt ',
        'sấu': u' xấu ','gut': u' tốt ', u' tot ': u' tốt ', u' nice ': u' tốt ', 'perfect': 'rất tốt', 'bt': u' bình thường ',
        'time': u' thời gian ', 'qá': u' quá ', u' ship ': u' giao hàng ', u' m ': u' mình ', u' mik ': u' mình ',
        'ể': 'ể', 'product': 'sản phẩm', 'quality': 'chất lượng','chat':' chất ', 'excelent': 'hoàn hảo', 'bad': 'tệ','fresh': ' tươi ','sad': ' tệ ',
        'date': u' hạn sử dụng ', 'hsd': u' hạn sử dụng ','quickly': u' nhanh ', 'quick': u' nhanh ','fast': u' nhanh ','delivery': u' giao hàng ',u' síp ': u' giao hàng ',
        'beautiful': u' đẹp tuyệt vời ', u' tl ': u' trả lời ', u' r ': u' rồi ', u' shopE ': u' cửa hàng ',u' order ': u' đặt hàng ',
        'chất lg': u' chất lượng ',u' sd ': u' sử dụng ',u' dt ': u' điện thoại ',u' nt ': u' nhắn tin ',u' tl ': u' trả lời ',u' sài ': u' xài ',u'bjo':u' bao giờ ',
        'thik': u' thích ',u' sop ': u' cửa hàng ', ' fb ': ' facebook ', ' face ': ' facebook ', ' very ': u' rất ',u'quả ng ':u' quảng  ',
        'dep': u' đẹp ',u' xau ': u' xấu ','delicious': u' ngon ', u'hàg': u' hàng ', u'qủa': u' quả ',
        'iu': u' yêu ','fake': u' giả mạo ', 'trl': 'trả lời', '><': u' positive ',
        ' por ': u' tệ ',' poor ': u' tệ ', 'ib':u' nhắn tin ', 'rep':u' trả lời ',u'fback':' feedback ','fedback':' feedback ',
        'bik': u'biết', 'full': 'đầy', 'free':u'miễn phí', 'hôg':u'không','wán':u'quán', 'mìh':u'mình', 'thờng':u'thường', 'hk':u'không','nhưg':u'nhưng',
        'đág':u'đáng', 'nóg':u'nóng', 'đườg':u'đường','uốg':u'uống','lạh':u'lạnh', 'zô':u'vô','nhìu':u'nhiều', 'cx':u'cũng','nchung':u'nói chung',
        'comment':u'bình luận', 'bye':u'tạm biệt','khg':u'không', 'lìn':u'negative',
        
        }
    
    doc = doc.lower()
    # replace words
    for k, v in replace_list.items():
        doc = doc.replace(k, v)

    # remove duplicate characters
    doc = re.sub(r'([A-Z])\1+', lambda m: m.group(1).lower(), doc, flags=re.IGNORECASE)

    # remove number, special character
    regex = "[^a-z0-9A-ZÀÁÂÃÈÉÊÌÍÒÓÔÕÙÚĂĐĨŨƠàáâãèéêìíòóôõùúăđĩũơƯĂẠẢẤẦẨẪẬẮẰẲẴẶẸẺẼỀỀỂưăạảấầẩẫậắằẳẵặẹẻẽềếểỄỆỈỊỌỎỐỒỔỖỘỚỜỞỠỢỤỦỨỪễệỉịọỏốồổỗộớờởỡợụủứừỬỮỰỲỴÝỶỸửữựỳỵỷỹ]"
    doc = re.sub(regex, " ", doc)
    doc = ViTokenizer.tokenize(doc)
    
    return doc


In [ ]:
print(df_train.iloc[1, 0])
preprocessing(df_train.iloc[1, 0])

Thứ 6 nào ta cùng quẩy 💣 💣 💣
 Vuvuzela beer club chung hệ thống Gogi House , Daruma , Kichi kichi nên giá cũng chưa 10% VAT . Thanh toán trước 20g hằng ngày đc giảm 30% thức ăn . Tính ra đc giảm 20% . Cũng zui ! 😊 😊 😊
 Ngồi thư giãn cũng ok . Thức ăn cũng ổn . Có mấy món nguyên liệu bên ngoài hok bik chỗ nào bán nên vô đây ăn . Cơ mà gọi mấy món đó cũng ko có hàng 😂 😂 😂
 Cháo bồ câu hình chụp nguyên con bồ câu mà tìm hoài chỉ thấy phao câu không thấy đầu bồ câu . Hỏi e nv e chạy hỏi nhà bếp đc trả lời là 1 phần cháo đc tính toán 1 lượng thịt cố định theo gram chứ không phải nguyên con chị ơi . 😅 😅 😅
 Ps : Lần sau sẽ rủ trên 4 đứa đi quẩy để đc tặng 10 ly biaaaaaaaaaaaaaa mí đc 🍺 🍺 🍺 🍺 🍺 🍺 🍺 🍺 🍺 🍺



'thứ 6 nào ta cùng quẩy vuvuzela ber club chung hệ_thống gogi house daruma kichi kichi nên giá cũng chưa 10 vat thanh_toán trước 20g hằng ngày được giảm 30 thức_ăn tính ra được giảm 20 cũng zui positive positive positive ngồi thư giãn cũng ok thức_ăn cũng ổn có mấy món nguyên_liệu bên ngoài không biết chỗ nào bán nên vô đây ăn cơ_mà gọi mấy món đó cũng không có hàng positive positive positive cháo bồ_câu hình chụp nguyên con bồ_câu mà tìm hoài chỉ thấy phao_câu không thấy đầu bồ_câu hỏi e nv e chạy hỏi nhà_bếp được trả_lời là 1 phần cháo được tính_toán 1 lượng thịt cố_định theo gram chứ không phải nguyên con chị ơi ps lần sau sẽ rủ trên 4 đứa đi quẩy để được tặng 10 ly bia mí được'

In [ ]:
df_train['comment'] = df_train['comment'].apply(preprocessing)
df_test['comment'] = df_test['comment'].apply(preprocessing)
df_train.head()

,comment,label
0,mua có mỗi bingsu thập_cẩm 45k mà mình f đợi h...,0
1,thứ 6 nào ta cùng quẩy vuvuzela ber club chung...,0
2,mình đi với nhóm tổng_cộng 4 người ăn chỉ có k...,0
3,nhân_viên phục_vụ không mấy tận_tình đồ ăn ra ...,0
4,vào đây thì hết bàn nhưng mình vẫn ngồi đợi bì...,0


In [ ]:
df_train.to_csv("/content/drive/MyDrive/Colab Notebooks/Sentiment_BML/train_processed.csv", index=False)
df_test.to_csv("/content/drive/MyDrive/Colab Notebooks/Sentiment_BML/test_processed.csv", index=False)


# Training



In [ ]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.metrics import classification_report
from sklearn.svm import LinearSVC
from sklearn.naive_bayes import MultinomialNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split

## Load Data

In [ ]:
# FOOODY 
import pandas as pd
import sklearn
df_train = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/Sentiment_BML/train_processed.csv")
df_test = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/Sentiment_BML/test_processed.csv")

X, Y = df_train['comment'], df_train['label']
X_test, Y_test = df_test['comment'], df_test['label']
print(X.shape)
print(X_test.shape)

(40000,)
(10000,)


## Base line

### BOW - RandomForest

In [ ]:
#Bow - Random Forest : base line: FOODY
count_vec = CountVectorizer()
pipeline = Pipeline(steps=[('vec', CountVectorizer()), ('clf', RandomForestClassifier())], verbose=True)

pipeline.fit(X, Y)
Y_pred = pipeline.predict(X_test)
print(classification_report(Y_test, Y_pred, digits=4))

[Pipeline] ............... (step 1 of 2) Processing vec, total=   2.0s
[Pipeline] ............... (step 2 of 2) Processing clf, total= 1.5min
              precision    recall  f1-score   support

           0     0.8613    0.8828    0.8719      5000
           1     0.8798    0.8578    0.8687      5000

    accuracy                         0.8703     10000
   macro avg     0.8705    0.8703    0.8703     10000
weighted avg     0.8705    0.8703    0.8703     10000



### TFIDF - Random Forest

In [ ]:
#TFIDF - RandomForest : Base line : FOODY

# max_df: means "ignore terms that appear in more than 50% of the documents".
# min_df:  means "ignore terms that appear in less than 5 documents".
pipeline = Pipeline(steps=[('vec', CountVectorizer()), 
                           ('tfidf', TfidfTransformer()),
                           ('clf', RandomForestClassifier())],
                             verbose=True
                    )

pipeline.fit(X, Y)
Y_pred = pipeline.predict(X_test)
print(classification_report(Y_test, Y_pred, digits=4))


[Pipeline] ............... (step 1 of 3) Processing vec, total=   2.0s
[Pipeline] ............. (step 2 of 3) Processing tfidf, total=   0.1s
[Pipeline] ............... (step 3 of 3) Processing clf, total= 1.3min
              precision    recall  f1-score   support

           0     0.8703    0.8708    0.8705      5000
           1     0.8707    0.8702    0.8705      5000

    accuracy                         0.8705     10000
   macro avg     0.8705    0.8705    0.8705     10000
weighted avg     0.8705    0.8705    0.8705     10000



## FINETUNING

### BOW - Random Forest - Optimal

In [ ]:
pipeline = Pipeline(steps=[('vec', CountVectorizer(ngram_range=(1,5),stop_words=None,max_df=0.5, min_df=5, )), ('clf', RandomForestClassifier())], verbose=True)
pipeline.fit(X, Y)
Y_pred = pipeline.predict(X_test)
print(classification_report(Y_test, Y_pred, digits=4))

[Pipeline] ............... (step 1 of 2) Processing vec, total=  57.7s
[Pipeline] ............... (step 2 of 2) Processing clf, total= 2.7min
              precision    recall  f1-score   support

           0     0.8841    0.8714    0.8777      5000
           1     0.8732    0.8858    0.8795      5000

    accuracy                         0.8786     10000
   macro avg     0.8787    0.8786    0.8786     10000
weighted avg     0.8787    0.8786    0.8786     10000



### TFIDF - Random Forest - Optimal

In [ ]:
pipeline = Pipeline(steps=[('vec', CountVectorizer(ngram_range=(1,5),stop_words=None,max_df=0.5, min_df=5, )), 
                           ('tfidf', TfidfTransformer(use_idf=True, sublinear_tf = True,norm='l2',smooth_idf=True)),
                           ('clf', RandomForestClassifier(n_estimators=150))],
                             verbose=True
                    )

pipeline.fit(X, Y)
Y_pred = pipeline.predict(X_test)
print(classification_report(Y_test, Y_pred, digits=4))

[Pipeline] ............... (step 1 of 3) Processing vec, total=  58.1s
[Pipeline] ............. (step 2 of 3) Processing tfidf, total=   0.5s
[Pipeline] ............... (step 3 of 3) Processing clf, total= 3.8min
              precision    recall  f1-score   support

           0     0.8943    0.8714    0.8827      5000
           1     0.8746    0.8970    0.8857      5000

    accuracy                         0.8842     10000
   macro avg     0.8845    0.8842    0.8842     10000
weighted avg     0.8845    0.8842    0.8842     10000



## Ensemble

In [ ]:
#  Ensemble Stacking: LinearSVC, RandomForest
from sklearn.ensemble import VotingClassifier, StackingClassifier
from sklearn.linear_model import LogisticRegression
estimators=[
                              ('nv', MultinomialNB(alpha=0.1)), 
                              ('rfc', RandomForestClassifier(n_estimators=300)),
                              ('svc',LinearSVC(C=1, verbose=True))
                              ]

clf = StackingClassifier(
    estimators=estimators, final_estimator=LogisticRegression(), stack_method='predict', verbose=True, n_jobs=-1
)


pipeline = Pipeline(steps=[('vec', CountVectorizer(ngram_range=(1,5),stop_words=None, min_df=5)), 
                           ('tfidf', TfidfTransformer(use_idf=True, sublinear_tf = True,norm='l2',smooth_idf=True)),
                        ('clf', clf)],
                             verbose=True
                    )



pipeline.fit(X, Y)
Y_pred = pipeline.predict(X_test)
print(classification_report(Y_test, Y_pred, digits=6))

[Pipeline] ............... (step 1 of 3) Processing vec, total= 1.6min
[Pipeline] ............. (step 2 of 3) Processing tfidf, total=   0.8s
[Pipeline] ............... (step 3 of 3) Processing clf, total=   7.9s
              precision    recall  f1-score   support

           0   0.913913  0.896000  0.904868      5000
           1   0.897999  0.915600  0.906714      5000

    accuracy                       0.905800     10000
   macro avg   0.905956  0.905800  0.905791     10000
weighted avg   0.905956  0.905800  0.905791     10000



### BOW - NaiveBayes